In [ ]:
# =====================================================================
# Phase 5 — LOSO Exporter ( BioRob)
# =====================================================================
from __future__ import annotations

import json
import math
import warnings
from pathlib import Path
from collections import OrderedDict, defaultdict

import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")

# ---------------- CONFIG ----------------

# NOTE: adjust ROOT_DIR so that DATASET_DIR matches what Phase 6 is using
ROOT_DIR    = Path(r"/home/tsultan1/BioRob(Final)/Data")
DATASET_DIR = ROOT_DIR / "_dataset_icml_v1"

SPLITS_CSV  = DATASET_DIR / "splits_v1.csv"
QC_CSV      = DATASET_DIR / "qc_summary_v1.csv"   # NEW: QC summary from Phase 4

# We now support TWO export modes:
#   1) balanced_supervised → for Phase 6 supervised LOSO (ICML + BioRob)
#   2) ssl_unbalanced      → for SSL pretraining (ICML + BioRob)
OUT_NAME_BALANCED = "exports_v1_balanced"   # indicates train balancing
OUT_NAME_SSL      = "exports_v1_ssl"        # no balancing, full distribution

EXPORT_BALANCED   = True
EXPORT_SSL        = True

# Window types used in splits_v1.csv
WINDOW_TYPES = ["sliding", "onset_anchor"]

# Modalities to include
INCLUDE_EEG, INCLUDE_EMG, INCLUDE_ET = True, True, True

# Shard size (number of windows per NPZ)
SHARD_SIZE   = 5000

# Fixed window length in samples (must match Phase 4 windowing, e.g. 500 @ 250 Hz = 2 s)
FIXED_WINDOW_LEN = 500

# Good coverage threshold for masks
MIN_COVERAGE = 0.40

# ---------------- TASK BALANCING CONFIG ----------------
# Used only for TRAIN within each fold, in BALANCED mode
#   - Limit Task 0 (rest) to at most 20% of TRAIN
#   - Make all non-zero tasks have the same number of samples in TRAIN
#     (over/under-sampling to a shared target_per_action)
MAX_TASK0_RATIO  = 0.20
MIN_TASK_SAMPLES = 50
ACTION_MAX_FACTOR = 3   # max per-action cap = ACTION_MAX_FACTOR * MIN_TASK_SAMPLES

# Optional caps for quick dry-runs (leave None for full dataset)
LIMIT_TRAIN = LIMIT_VAL = LIMIT_TEST = None

# ---------------- ANALYSIS / BALANCING HELPERS ----------------
def analyze_task_distribution(splits_df: pd.DataFrame):
    """Analyze and display the current overall task distribution."""
    print("\n=== CURRENT TASK DISTRIBUTION ANALYSIS (ALL SPLITS AFTER QC) ===")

    total_windows = len(splits_df)
    task_dist = splits_df["task_target"].value_counts().sort_index()

    print(f"Total windows: {total_windows}")
    print("\nTask distribution (task_target):")
    for task, count in task_dist.items():
        percentage = count / total_windows * 100
        print(f"  Task {task}: {count} samples ({percentage:.1f}%)")

    task0_ratio = task_dist.get(0, 0) / max(total_windows, 1)
    if task0_ratio > MAX_TASK0_RATIO:
        print(f"\n⚠️  WARNING: Task 0 (rest) dominates with {task0_ratio:.1%} of samples")
        print(f"   Target maximum for Task 0 in TRAIN (balanced export): {MAX_TASK0_RATIO:.1%}")

    return task_dist

def rebalance_split(split_data: pd.DataFrame, is_train: bool = True) -> pd.DataFrame:
    """
    Rebalance a single split (train/val/test).
    Used ONLY for TRAIN within each fold in the BALANCED supervised export.

    New strategy (for TRAIN):
      - Make all non-zero tasks (actions) have the SAME number of samples:
          target_per_action = clip(max_count, [MIN_TASK_SAMPLES, MIN_TASK_SAMPLES * ACTION_MAX_FACTOR])
        using over/under-sampling per action.

      - Then set Task 0 (rest) to MAX_TASK0_RATIO of (rest + actions) via:
          target_task0 = total_other_balanced * r / (1 - r)

    VAL/TEST currently never call this (we keep their original distribution).
    """
    task0_data       = split_data[split_data["task_target"] == 0]
    other_tasks_data = split_data[split_data["task_target"] != 0]

    if is_train:
        target_task0_ratio = MAX_TASK0_RATIO
        min_task_samples   = MIN_TASK_SAMPLES
    else:
        # Not really used now (we don't rebalance val/test), but kept for completeness
        target_task0_ratio = min(MAX_TASK0_RATIO + 0.1, 0.3)
        min_task_samples   = max(10, MIN_TASK_SAMPLES // 2)

    if len(other_tasks_data) == 0:
        # Degenerate case: only Task 0 present
        return split_data.sample(frac=1, random_state=42).reset_index(drop=True)

    # --- Step 1: equalize all non-zero action classes ---
    other_task_dist = other_tasks_data["task_target"].value_counts().sort_index()
    max_cur = other_task_dist.max()
    max_action_samples = min_task_samples * ACTION_MAX_FACTOR

    # Shared target per action:
    #   - at least MIN_TASK_SAMPLES
    #   - at most min(max_cur, MAX_ACTION_SAMPLES)
    target_per_action = max(min_task_samples, min(max_cur, max_action_samples))

    print("  [rebalance_split] Action-task distribution BEFORE balancing:")
    for t, c in other_task_dist.items():
        print(f"    Task {t}: {c}")

    print(f"  [rebalance_split] Using target_per_action = {target_per_action} per non-zero task")

    balanced_other = []
    for task in other_task_dist.index:
        task_data = other_tasks_data[other_tasks_data["task_target"] == task]
        cur = len(task_data)

        if cur < target_per_action:
            # Oversample up to target_per_action
            balanced_task = task_data.sample(
                n=target_per_action, random_state=42, replace=True
            )
        elif cur > target_per_action:
            # Undersample down to target_per_action
            balanced_task = task_data.sample(
                n=target_per_action, random_state=42, replace=False
            )
        else:
            balanced_task = task_data

        balanced_other.append(balanced_task)

    all_balanced_other = pd.concat(balanced_other, ignore_index=True)
    total_other_balanced = len(all_balanced_other)

    # --- Step 2: set Task 0 (rest) to respect MAX_TASK0_RATIO ---
    if (1.0 - target_task0_ratio) > 0.0:
        target_task0 = int(total_other_balanced * target_task0_ratio / (1.0 - target_task0_ratio))
    else:
        target_task0 = len(task0_data)

    print(f"  [rebalance_split] total_other_balanced = {total_other_balanced}")
    print(f"  [rebalance_split] target_task0 (rest) = {target_task0} (ratio ≈ {target_task0_ratio:.2f})")

    if len(task0_data) < target_task0:
        # Oversample rest
        task0_balanced = task0_data.sample(
            n=target_task0, random_state=42, replace=True
        )
    elif len(task0_data) > target_task0:
        # Undersample rest
        task0_balanced = task0_data.sample(
            n=target_task0, random_state=42, replace=False
        )
    else:
        task0_balanced = task0_data

    # Concatenate rest + balanced actions
    split_balanced = pd.concat([task0_balanced, all_balanced_other], ignore_index=True)

    # Shuffle
    split_balanced = split_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

    # Debug print AFTER balancing
    dist = split_balanced["task_target"].value_counts().sort_index()
    print("  [rebalance_split] New TRAIN distribution (after balancing):")
    for t, c in dist.items():
        print(f"    Task {t}: {c}")

    return split_balanced

# ---------------- ORIGINAL PHASE 5 FUNCTIONS (mostly unchanged) ----------------
EEG_KEY      = "EEG"
EEG_MASK_KEY = "EEG_mask"
EEG_CH_KEY   = "EEG_ch"

EMG_KEY      = "EMG_env"
EMG_MASK_KEY = "EMG_mask"
EMG_CH_KEY   = "EMG_ch"

ET_KEY       = "ET"
ET_MASK_KEY  = "ET_mask"
ET_CH_KEY    = "ET_ch"

NPZ_SUFFIX = ".preproc.npz"

def _ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

def _load_npz(npz_path: Path) -> dict:
    with np.load(npz_path, allow_pickle=True) as z:
        out = {k: z[k] for k in z.files}
    # convert object arrays of channel names back to list[str]
    for k in (EEG_CH_KEY, EMG_CH_KEY, ET_CH_KEY):
        if k in out:
            out[k] = [str(x) for x in out[k].tolist()]
    return out

class NPZCacheLRU:
    def __init__(self, capacity=16):
        self.capacity = int(capacity)
        self._d: OrderedDict[str, dict] = OrderedDict()
    def get(self, csv_path: str):
        key = csv_path + NPZ_SUFFIX
        if key in self._d:
            self._d.move_to_end(key)
            return self._d[key]
        npz_path = Path(csv_path + NPZ_SUFFIX)
        if not npz_path.exists():
            raise FileNotFoundError(f"cache missing: {npz_path}")
        val = _load_npz(npz_path)
        self._d[key] = val
        self._d.move_to_end(key)
        if len(self._d) > self.capacity:
            self._d.popitem(last=False)
        return val
    def clear(self):
        self._d.clear()

def _apply_window(arr: np.ndarray, s: int, e: int) -> np.ndarray:
    s = max(0, int(s)); e = min(int(e), arr.shape[0])
    if e <= s:
        return arr[:0]
    return arr[s:e]

def _accumulate_stats(stats, X: np.ndarray, M: np.ndarray):
    """Accumulate per-channel sums/sumsq/count across time using mask (0/1)."""
    if X.size == 0:
        return
    # Extra guard: avoid any NaNs/Infs sneaking into stats
    X = np.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0)
    m = M.astype(np.float32)
    cnt = m.sum(axis=0)                 # (C,)
    if cnt.sum() == 0:
        return
    stats["cnt"]   += cnt
    stats["sum"]   += (X * m).sum(axis=0)
    stats["sum2"]  += ((X ** 2) * m).sum(axis=0)

def _finalize_stats(stats):
    cnt  = np.maximum(stats["cnt"], 1.0)
    mean = stats["sum"] / cnt
    var  = np.maximum(stats["sum2"] / cnt - mean**2, 0.0)
    std  = np.sqrt(var) + 1e-8  # Slightly larger epsilon for stability
    return mean.astype(np.float32), std.astype(np.float32)

def _new_stats(nc: int):
    return {"cnt":  np.zeros(nc, dtype=np.float64),
            "sum":  np.zeros(nc, dtype=np.float64),
            "sum2": np.zeros(nc, dtype=np.float64)}

def _normalize(X: np.ndarray, M: np.ndarray, mean: np.ndarray, std: np.ndarray):
    if X.size == 0:
        return X.astype(np.float32)
    Y = (X - mean[None, :]) / std[None, :]
    # where mask==0, set to 0 to avoid injecting arbitrary values
    Y[M <= 0.0] = 0.0
    # Hard guard: no NaN/Inf going to model
    Y = np.nan_to_num(Y, nan=0.0, posinf=0.0, neginf=0.0)
    return Y.astype(np.float32)

def _coverage_ok(M: np.ndarray, thr: float) -> bool:
    if M.size == 0:
        return True
    cov = M.mean(axis=0)  # coverage per channel
    return bool((cov >= thr).mean() > 0.5)  # majority of channels OK

def _row_limit(split: str):
    if split == "train" and LIMIT_TRAIN:
        return LIMIT_TRAIN
    if split == "val"   and LIMIT_VAL:
        return LIMIT_VAL
    if split == "test"  and LIMIT_TEST:
        return LIMIT_TEST
    return None

def _pad_or_trim_time(X: np.ndarray, target_len: int) -> np.ndarray:
    """
    Ensure X has shape (target_len, C):
      - If longer, truncate.
      - If shorter, pad with zeros at the end.
    """
    if X is None or X.size == 0:
        return np.zeros((target_len, 0), dtype=np.float32)

    T, C = X.shape
    if T == target_len:
        return X
    if T > target_len:
        return X[:target_len]
    # T < target_len → pad
    pad = np.zeros((target_len - T, C), dtype=X.dtype)
    return np.concatenate([X, pad], axis=0)

# ---------------- EXPORT CORE ----------------
def export_fold(splits: pd.DataFrame, fold_id: int, out_root: Path, cache_cap=12, mode: str = "balanced"):
    """
    Export one fold in either:
        mode = "balanced"  → supervised, with TRAIN balancing
        mode = "ssl"       → SSL, NO balancing (original splits_v1 distribution)
    Labels are ALWAYS exported (y_action, y_task) in both modes.
    """
    if mode not in ("balanced", "ssl"):
        raise ValueError(f"Unknown mode: {mode}")

    is_balanced = (mode == "balanced")
    out_name    = OUT_NAME_BALANCED if is_balanced else OUT_NAME_SSL

    mode_label = "BALANCED (supervised)" if is_balanced else "SSL (unbalanced)"
    print(f"\n========== FOLD {fold_id} — export mode: {mode_label} ==========")

    fold_rows = splits[splits["fold_id"] == fold_id]
    if fold_rows.empty:
        print(f"[skip] fold {fold_id}: no rows")
        return

    # Keep only desired window types
    fold_rows = fold_rows[fold_rows["type"].isin(WINDOW_TYPES)]
    if fold_rows.empty:
        print(f"[skip] fold {fold_id}: no rows after WINDOW_TYPES filter {WINDOW_TYPES}")
        return

    cache = NPZCacheLRU(cache_cap)

    # ---- PASS 1: accumulate TRAIN stats (UNBALANCED) ----
    tr_rows_stats = fold_rows[fold_rows["split"] == "train"]
    if tr_rows_stats.empty:
        print(f"[warn] fold {fold_id}: no TRAIN rows; skipping")
        return

    # limit for speed if requested
    lim = _row_limit("train")
    if lim:
        tr_rows_stats = tr_rows_stats.iloc[:lim]

    # Analyze fold TRAIN distribution before processing
    print(f"[fold {fold_id}] Pre-export TRAIN task distribution (mode={mode}):")
    fold_dist = tr_rows_stats["task_target"].value_counts().sort_index()
    for task, count in fold_dist.items():
        print(f"  Task {task}: {count} samples")

    # Determine channel counts from a sample cache
    stats = {}
    def _ensure_stat(key, nchan):
        if key not in stats:
            stats[key] = _new_stats(nchan)

    print(f"[fold {fold_id}] pass1: accumulating TRAIN stats over {len(tr_rows_stats):,} windows …")
    for r in tr_rows_stats.itertuples(index=False):
        npz = cache.get(r.file)
        s, e = int(r.start_idx), int(r.end_idx)

        # EEG
        if INCLUDE_EEG and EEG_KEY in npz and EEG_MASK_KEY in npz:
            Xe = _apply_window(npz[EEG_KEY], s, e)
            Me = _apply_window(npz[EEG_MASK_KEY], s, e)
            if Xe.size and _coverage_ok(Me, MIN_COVERAGE):
                _ensure_stat(EEG_KEY, Xe.shape[1])
                _accumulate_stats(stats[EEG_KEY], Xe, Me)

        # EMG
        if INCLUDE_EMG and EMG_KEY in npz and EMG_MASK_KEY in npz:
            Xm = _apply_window(npz[EMG_KEY], s, e)
            Mm = _apply_window(npz[EMG_MASK_KEY], s, e)
            if Xm.size and _coverage_ok(Mm, MIN_COVERAGE):
                _ensure_stat(EMG_KEY, Xm.shape[1])
                _accumulate_stats(stats[EMG_KEY], Xm, Mm)

        # ET
        if INCLUDE_ET and ET_KEY in npz and ET_MASK_KEY in npz:
            Xt = _apply_window(npz[ET_KEY], s, e)
            Mt = _apply_window(npz[ET_MASK_KEY], s, e)
            if Xt.size and _coverage_ok(Mt, MIN_COVERAGE):
                _ensure_stat(ET_KEY, Xt.shape[1])
                _accumulate_stats(stats[ET_KEY], Xt, Mt)

    # Finalize stats
    means, stds = {}, {}
    for key in (EEG_KEY, EMG_KEY, ET_KEY):
        if key in stats:
            m, s = _finalize_stats(stats[key])
            means[key], stds[key] = m, s

    # Save per-fold stats (channel names included)
    fold_dir = out_root / f"{out_name}_fold{fold_id}"
    _ensure_dir(fold_dir)
    meta = {
        "fold_id": int(fold_id),
        "export_mode": mode,   # "balanced" or "ssl"
        "window_types": WINDOW_TYPES,
        "fixed_window_len": FIXED_WINDOW_LEN,
        "min_coverage": MIN_COVERAGE,
        "include": {"EEG": INCLUDE_EEG, "EMG": INCLUDE_EMG, "ET": INCLUDE_ET},
        "balancing_info": {},
        "channels": {},
        "stats": {}
    }
    if is_balanced:
        meta["balancing_info"] = {
            "max_task0_ratio_train": MAX_TASK0_RATIO,
            "min_task_samples_train": MIN_TASK_SAMPLES,
            "action_max_factor": ACTION_MAX_FACTOR,
            "note": (
                "Balancing applied ONLY to TRAIN split in this export: "
                "all non-zero tasks are equalized to target_per_action; "
                "Task 0 (rest) capped via MAX_TASK0_RATIO."
            ),
        }
    else:
        meta["balancing_info"] = {
            "note": "NO within-fold balancing; keeps original splits_v1 distribution."
        }

    # get channel lists from any cached file
    sample_npz = cache.get(tr_rows_stats.iloc[0].file)
    if INCLUDE_EEG and EEG_CH_KEY in sample_npz:
        meta["channels"]["EEG"] = sample_npz[EEG_CH_KEY]
    if INCLUDE_EMG and EMG_CH_KEY in sample_npz:
        meta["channels"]["EMG"] = sample_npz[EMG_CH_KEY]
    if INCLUDE_ET  and ET_CH_KEY  in sample_npz:
        meta["channels"]["ET"]  = sample_npz[ET_CH_KEY]
    for key in (EEG_KEY, EMG_KEY, ET_KEY):
        if key in means:
            meta["stats"][key] = {"mean": means[key].tolist(), "std": stds[key].tolist()}
    (fold_dir / "stats_fold.json").write_text(json.dumps(meta, indent=2))

    # ---- PASS 2: write normalized shards for all splits ----
    for split in ["train", "val", "test"]:
        rows = fold_rows[fold_rows["split"] == split]
        if rows.empty:
            print(f"[fold {fold_id}] no rows for {split}")
            continue
        lim = _row_limit(split)
        if lim:
            rows = rows.iloc[:lim]

        # BALANCED export: apply balancing ONLY to TRAIN
        # SSL export: NEVER balance, keep original distribution
        if split == "train" and is_balanced:
            print(f"[fold {fold_id}] applying within-fold TRAIN balancing (mode=balanced) …")
            rows = rebalance_split(rows, is_train=True)
        else:
            print(f"[fold {fold_id}] keeping original {split} distribution (mode={mode}, no balancing)")

        out_split = fold_dir / split
        _ensure_dir(out_split)

        # reset shard buffers
        shard_idx = 1
        buf = defaultdict(list)  # keys: X_EEG, X_EMG, X_ET, y_action, y_task, meta
        total_windows = 0

        def _flush():
            nonlocal shard_idx, buf
            if not buf["y_action"]:  # empty shard
                return
            shard_path = out_split / f"{split}_shard_{shard_idx:04d}.npz"
            np.savez_compressed(
                shard_path,
                # Stacked arrays per modality (N, T, C)
                X_EEG=np.stack(buf["X_EEG"]) if buf["X_EEG"] else np.zeros((0, 1, 0), np.float32),
                X_EMG=np.stack(buf["X_EMG"]) if buf["X_EMG"] else np.zeros((0, 1, 0), np.float32),
                X_ET =np.stack(buf["X_ET" ]) if buf["X_ET" ] else np.zeros((0, 1, 0), np.float32),
                # Labels (still present in SSL; SSL just ignores them at training time)
                y_action = np.asarray(buf["y_action"], dtype=np.int8),
                y_task   = np.asarray(buf["y_task"],   dtype=np.int16),
                # Optional metadata (can help debugging or SSL sampling)
                subject_id = np.asarray(buf["subject_id"], dtype=np.int16),
                task_code  = np.asarray(buf["task_code"],  dtype=np.int16),
                trial_id   = np.asarray(buf["trial_id"],   dtype=np.int16),
                win_type   = np.asarray(buf["win_type"],   dtype=object),
                # Save window lengths for safety
                win_len    = np.asarray(buf["win_len"],    dtype=np.int32),
            )
            shard_idx += 1
            buf = defaultdict(list)

        print(f"[fold {fold_id}] pass2: exporting {split} ({len(rows):,} windows, mode={mode}) …")

        # Print split distribution (after balancing for TRAIN if mode=balanced)
        split_dist = rows["task_target"].value_counts().sort_index()
        print(f"  {split} distribution (task_target):")
        for task, count in split_dist.items():
            print(f"    Task {task}: {count} samples")

        for r in rows.itertuples(index=False):
            npz = cache.get(r.file)
            s, e = int(r.start_idx), int(r.end_idx)
            win_len = max(0, e - s)  # original logical length (for metadata only)

            # Modality windows (T, C)
            Xe = Me = Xm = Mm = Xt = Mt = None

            if INCLUDE_EEG and EEG_KEY in npz and EEG_MASK_KEY in npz and EEG_KEY in means:
                Xe = _apply_window(npz[EEG_KEY], s, e)
                Me = _apply_window(npz[EEG_MASK_KEY], s, e)
                if Xe.size == 0 or not _coverage_ok(Me, MIN_COVERAGE):
                    Xe = None
                else:
                    Xe = _normalize(Xe, Me, means[EEG_KEY], stds[EEG_KEY])

            if INCLUDE_EMG and EMG_KEY in npz and EMG_MASK_KEY in npz and EMG_KEY in means:
                Xm = _apply_window(npz[EMG_KEY], s, e)
                Mm = _apply_window(npz[EMG_MASK_KEY], s, e)
                if Xm.size == 0 or not _coverage_ok(Mm, MIN_COVERAGE):
                    Xm = None
                else:
                    Xm = _normalize(Xm, Mm, means[EMG_KEY], stds[EMG_KEY])

            if INCLUDE_ET and ET_KEY in npz and ET_MASK_KEY in npz and ET_KEY in means:
                Xt = _apply_window(npz[ET_KEY], s, e)
                Mt = _apply_window(npz[ET_MASK_KEY], s, e)
                if Xt.size == 0 or not _coverage_ok(Mt, MIN_COVERAGE):
                    Xt = None
                else:
                    Xt = _normalize(Xt, Mt, means[ET_KEY], stds[ET_KEY])

            # If no modality survives coverage/stat checks, skip this window
            if (Xe is None) and (Xm is None) and (Xt is None):
                continue

            # Enforce FIXED_WINDOW_LEN for all modalities
            T_len = FIXED_WINDOW_LEN

            def _pad_trim_if_not_none(X):
                if X is None:
                    return None
                return _pad_or_trim_time(X, T_len)

            Xe = _pad_trim_if_not_none(Xe)
            Xm = _pad_trim_if_not_none(Xm)
            Xt = _pad_trim_if_not_none(Xt)

            # Stack modality dims to match (T_len, C). Use empty (T_len, 0) if absent.
            if Xe is None:
                Xe = np.zeros((T_len, 0), np.float32)
            if Xm is None:
                Xm = np.zeros((T_len, 0), np.float32)
            if Xt is None:
                Xt = np.zeros((T_len, 0), np.float32)

            # Extra safety: ensure no NaN/Inf after normalization + padding
            Xe = np.nan_to_num(Xe, nan=0.0, posinf=0.0, neginf=0.0)
            Xm = np.nan_to_num(Xm, nan=0.0, posinf=0.0, neginf=0.0)
            Xt = np.nan_to_num(Xt, nan=0.0, posinf=0.0, neginf=0.0)

            # Append to buffers
            buf["X_EEG"].append(Xe)
            buf["X_EMG"].append(Xm)
            buf["X_ET"].append(Xt)
            buf["y_action"].append(int(r.label_action))
            buf["y_task"].append(int(r.task_target))
            buf["subject_id"].append(int(r.subject_id))
            buf["task_code"].append(int(r.task_code))
            buf["trial_id"].append(int(r.trial_id))
            buf["win_type"].append(str(r.type))
            buf["win_len"].append(win_len)  # original (may differ from FIXED_WINDOW_LEN)
            total_windows += 1

            if len(buf["y_action"]) >= SHARD_SIZE:
                _flush()

        _flush()  # final shard
        # Write a tiny split-level meta
        (out_split / "split_meta.json").write_text(json.dumps({
            "num_windows": int(total_windows),
            "shard_size": SHARD_SIZE,
            "limit": _row_limit(split),
            "task_distribution": rows["task_target"].value_counts().sort_index().to_dict(),
            "export_mode": mode,
            "fixed_window_len": FIXED_WINDOW_LEN
        }, indent=2))

    cache.clear()
    print(f"[fold {fold_id}] done → {out_name}_fold{fold_id} (mode={mode})")

# ---------------- MAIN ----------------
if __name__ == "__main__":
    if not SPLITS_CSV.exists():
        raise SystemExit(f"[stop] splits not found: {SPLITS_CSV}")

    print("Loading splits data...")
    splits = pd.read_csv(SPLITS_CSV)

    # ---------------- APPLY QC FILTER (Phase 4 → Phase 5 bridge) ----------------
    if QC_CSV.exists():
        print(f"\n[QC] Loading QC summary from: {QC_CSV}")
        qc = pd.read_csv(QC_CSV)

        # Normalise to a "keep" mask
        if "qc_keep" in qc.columns:
            qc_use = qc[qc["qc_keep"] == 1].copy()
            print(f"[QC] Found 'qc_keep' column → keeping {len(qc_use)}/{len(qc)} files")
        elif "qc_flag" in qc.columns:
            # Treat 'bad' (or similar) as reject; others as keep
            bad_vals = {"bad", "reject", "fail", "drop"}
            qc["qc_flag_str"] = qc["qc_flag"].astype(str).str.lower()
            qc_use = qc[~qc["qc_flag_str"].isin(bad_vals)].copy()
            print(f"[QC] Found 'qc_flag' column → keeping {len(qc_use)}/{len(qc)} files")
        else:
            qc_use = qc.copy()
            print("[QC] No 'qc_keep' or 'qc_flag' columns found → treating all QC entries as keep")

        # Join by filename (basename) to be robust to absolute/relative paths
        splits["file_base"] = splits["file"].apply(lambda x: Path(x).name)
        qc_use["file_base"] = qc_use["file"].apply(lambda x: Path(x).name)

        good_files = set(qc_use["file_base"].unique())
        before = len(splits)
        splits = splits[splits["file_base"].isin(good_files)].reset_index(drop=True)
        after = len(splits)
        dropped = before - after

        print(f"[QC] Applied QC filter to splits_v1:")
        print(f"     windows before: {before}")
        print(f"     windows after : {after}")
        print(f"     windows dropped (qc_bad files): {dropped}\n")

        splits.drop(columns=["file_base"], inplace=True)
    else:
        print(f"\n[QC] qc_summary_v1.csv not found at {QC_CSV}")
        print("     Proceeding without per-file QC filtering.\n")

    # Basic sanity check
    must = {
        "file", "subject_id", "split", "type", "task_target",
        "label_action", "start_idx", "end_idx", "task_code",
        "trial_id", "fold_id"
    }
    missing = must - set(splits.columns)
    if missing:
        raise SystemExit(f"[stop] splits missing columns: {sorted(missing)}")

    # Analyze original distribution (logging only, after QC)
    analyze_task_distribution(splits)

    # Export folds
    folds = sorted(splits["fold_id"].unique().tolist())
    out_root = DATASET_DIR

    print(f"\nExporting {len(folds)} folds...")
    for k in folds:
        k = int(k)
        if EXPORT_BALANCED:
            export_fold(splits, fold_id=k, out_root=out_root, cache_cap=12, mode="balanced")
        if EXPORT_SSL:
            export_fold(splits, fold_id=k, out_root=out_root, cache_cap=12, mode="ssl")

    print("\n🎉 Phase 5 complete!")
    if EXPORT_BALANCED:
        print(f"   Supervised (balanced) output prefix: {OUT_NAME_BALANCED}_fold*")
        print(f"   Train Task 0 target ratio: ≤{MAX_TASK0_RATIO:.1%}")
        print(f"   Minimum TRAIN samples per non-zero task: {MIN_TASK_SAMPLES}")
        print(f"   Non-zero tasks equalized with ACTION_MAX_FACTOR={ACTION_MAX_FACTOR}")
    if EXPORT_SSL:
        print(f"   SSL (unbalanced) output prefix: {OUT_NAME_SSL}_fold*")
        print("   SSL exports keep full original distribution and all labels.")
